# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities_df

,Unnamed: 0,City,Country,Latitude,Longitudes,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,vaini,TO,-21.2000,-175.2000,73.40,78,75,5.75,1618583553
1,3,kapaa,US,22.0752,-159.3190,66.20,88,20,5.75,1618583554
2,4,provideniya,RU,64.3833,-173.3000,28.06,99,99,9.17,1618583554
3,5,egvekinot,RU,66.3167,-179.1667,24.75,99,100,8.28,1618583554
4,7,alofi,NU,-19.0595,-169.9187,69.80,100,98,1.14,1618583554
...,...,...,...,...,...,...,...,...,...,...
539,593,ngunguru,NZ,-35.6167,174.5000,63.00,90,5,5.99,1618583636
540,594,russell,NZ,32.3502,-85.1999,55.40,62,90,2.59,1618583636
541,595,asau,TV,46.4333,26.4000,52.00,26,92,12.19,1618583636
542,597,beringovskiy,RU,63.0500,179.3167,28.65,91,100,11.01,1618583636


In [14]:
cities_df[cities_df["Latitude"]<-90]

,Unnamed: 0,City,Country,Latitude,Longitudes,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date


In [18]:
sam_locations = [(cities_df.iloc[i,3],cities_df.iloc[i,4]) for i in range(30,544,200)]
sam_weights = [cities_df.iloc[i,6] for i in range(30,544,200)]
print(sam_weights)
sam_locations 

[81, 16, 51]


[(37.6138, -122.4869), (-22.9575, 14.5053), (51.0333, -4.2167)]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(sam_locations,sam_weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
